In [ ]:
import asyncio
import aiohttp
import time
from urllib.parse import urljoin

class RateLimiter:
    def __init__(self, max_requests, interval):
        self.max_requests = max_requests
        self.interval = interval
        self.requests = 0
        self.last_request_time = time.time()

    async def allow_request(self):
        now = time.time()
        if now - self.last_request_time > self.interval:
            self.requests = 0
            self.last_request_time = now
        if self.requests < self.max_requests:
            self.requests += 1
            return True
        return False

async def generate_subfolder_url(parent_url, subfolder_id):
    return urljoin(parent_url, f'{subfolder_id}/')

async def fetch_url(session, url, limiter):
    allowed = await limiter.allow_request()
    if allowed:
        async with session.get(url) as response:
            return await response.json()
    else:
        print("Rate limit exceeded. Waiting...")
        await asyncio.sleep(1)  # Example: Wait for a second before retrying
        return await fetch_url(session, url, limiter)  # Retry recursively

async def fetch_folder_contents(session, folder_url, limiter):
    results = []
    contents = await fetch_url(session, folder_url, limiter)

    for item in contents['items']:
        if item['type'] == 'file':
            file_url = urljoin(folder_url, item['name'])
            results.append(await fetch_url(session, file_url, limiter))
        elif item['type'] == 'folder':
            subfolder_url = await generate_subfolder_url(folder_url, item['id'])
            results.extend(await fetch_folder_contents(session, subfolder_url, limiter))

    return results

async def main():
    base_url = 'https://api.example.com/'
    initial_folder = 'root_folder'
    limiter = RateLimiter(max_requests=5, interval=1)  # Rate limit: 5 requests per second

    async with aiohttp.ClientSession() as session:
        initial_folder_url = urljoin(base_url, initial_folder)
        results = await fetch_folder_contents(session, initial_folder_url, limiter)
        print(results)  # Process or store the fetched results as needed

asyncio.run(main())
